In [1]:
import pandas as pd
import pandasql as ps

Корабли в «классах» построены по одному и тому же проекту, и классу присваивается либо имя первого корабля, построенного по данному проекту, либо названию класса дается имя проекта, которое не совпадает ни с одним из кораблей в БД. Корабль, давший название классу, называется головным.
Отношение Classes содержит имя класса, тип (bb для боевого (линейного) корабля или bc для боевого крейсера), страну, в которой построен корабль, число главных орудий, калибр орудий (диаметр ствола орудия в дюймах) и водоизмещение ( вес в тоннах). В отношении Ships записаны название корабля, имя его класса и год спуска на воду. В отношение Battles включены название и дата битвы, в которой участвовали корабли, а в отношении Outcomes – результат участия данного корабля в битве (потоплен-sunk, поврежден - damaged или невредим - OK).
Замечания. 1) В отношение Outcomes могут входить корабли, отсутствующие в отношении Ships. 2) Потопленный корабль в последующих битвах участия не принимает.

![](scheme.png)

In [2]:
classes = pd.read_csv('classes.csv', index_col='class')
classes

,type,country,guns,bore,displacement
class,,,,,
Bismarck,bb,Germany,8,15.0,42000
Iowa,bb,USA,9,16.0,46000
Kongo,bc,Japan,8,14.0,32000
North Carolina,bb,USA,12,16.0,37000
Renown,bc,Gt.Britain,6,15.0,32000
Revenge,bb,Gt.Britain,8,15.0,29000
Tennessee,bb,USA,12,14.0,32000
Yamato,bb,Japan,9,18.0,65000


In [3]:
ships = pd.read_csv('ships.csv', index_col='name')
ships

,class,launched
name,,
California,Tennessee,1921
Haruna,Kongo,1916
Hiei,Kongo,1914
Iowa,Iowa,1943
Kirishima,Kongo,1915
Kongo,Kongo,1913
Missouri,Iowa,1944
Musashi,Yamato,1942
New Jersey,Iowa,1943


In [258]:
outcomes = pd.read_csv('outcomes.csv', index_col=['ship', 'battle'])
outcomes

,,result
ship,battle,
Bismarck,North Atlantic,sunk
California,Surigao Strait,OK
Duke of York,North Cape,OK
Fuso,Surigao Strait,sunk
Hood,North Atlantic,sunk
King George V,North Atlantic,OK
Kirishima,Guadalcanal,sunk
Prince of Wales,North Atlantic,damaged
Rodney,North Atlantic,OK


In [5]:
battles = pd.read_csv('battles.csv', index_col='name', parse_dates=['date'])
battles

,date
name,
Guadalcanal,1942-11-15
North Atlantic,1941-05-25
North Cape,1943-12-26
Surigao Strait,1944-10-25


### Задача 1
По Вашингтонскому международному договору от начала 1922 г. запрещалось строить линейные корабли водоизмещением более 35 тыс.тонн. Укажите корабли, нарушившие этот договор (учитывать только корабли с известным годом спуска на воду). Вывести названия кораблей.
![](scheme.png)

In [54]:
query = '''
select name
from ships
inner join (
    select class from classes
    where type = 'bb' and displacement > 35000
)
rulebreakers on ships.class = rulebreakers.class
where launched >= 1922
'''
ps.sqldf(query)

,name
0,Iowa
1,Missouri
2,Musashi
3,New Jersey
4,North Carolina
5,Washington
6,Wisconsin
7,Yamato
8,South Dakota


In [196]:
complete_table = pd.merge(ships, classes, right_index=True, left_on='class')
mask = (complete_table['type'] == 'bb') & \
        (complete_table['displacement'] > 35000) & \
        (complete_table['launched'] >= 1922)
result = complete_table.loc[mask, []].reset_index()
result

,name
0,Iowa
1,Missouri
2,New Jersey
3,Wisconsin
4,Musashi
5,Yamato
6,North Carolina
7,Washington
8,South Dakota


### Задача 2

Найдите корабли, "сохранившиеся для будущих сражений"; т.е. выведенные из строя в одной битве (damaged), они участвовали в другой, произошедшей позже. Здесь важно учесть дату.

![](scheme.png)

In [83]:
query = '''
select damaged_ships.ship from
(
    select ship, date as damage_date
    from outcomes, battles
    where result = 'damaged' and battle = name
) damaged_ships,
(
    select ship, max(date) as last_battle_date
    from outcomes, battles
    where battle = name
    group by ship
) last_battles
where damaged_ships.ship = last_battles.ship and last_battle_date > damage_date
'''
ps.sqldf(query)

,ship
0,California


In [190]:
everything = pd.merge(outcomes, battles, right_index=True, left_on='battle')
damaged_ships = everything.loc[outcomes['result'] == 'damaged'].max(level='ship')
damaged_ships
last_battle_of_each_ship = everything.drop(columns=['result']).max(level='ship')
last_battle_of_each_ship.columns = ['last_battle_date']
damaged_ships_present_and_future = pd.merge(last_battle_of_each_ship,
                                            damaged_ships,
                                            right_index=True, left_on='ship')
result = damaged_ships_present_and_future.loc[damaged_ships_present_and_future['last_battle_date'] > \
                                              damaged_ships_present_and_future['date'], []].reset_index()
result

,ship
0,California


### Задача 3
Укажите названия, водоизмещение и число орудий, кораблей участвовавших в сражении при Гвадалканале (Guadalcanal).
![](scheme.png)

In [274]:
query = '''
select outcomes.ship, displacement, guns
from outcomes, ships, classes
where battle = 'Guadalcanal' and ships.class = classes.class and outcomes.ship = name
'''
ps.sqldf(query)

,ship,displacement,guns
0,Kirishima,32000,8
1,South Dakota,37000,12
2,Washington,37000,12
3,California,32000,12


In [276]:
outcomes = outcomes.reset_index().set_index('ship')
ships_characteristics = pd.merge(ships, classes, right_index=True, left_on='class')
everything = pd.merge(outcomes, ships_characteristics, left_index=True, right_index=True)
result = everything.loc[everything['battle'] == 'Guadalcanal', ['displacement', 'guns']].reset_index()
result.columns = ['ship', 'displacement', 'guns']
result

,ship,displacement,guns
0,California,32000,12
1,Kirishima,32000,8
2,South Dakota,37000,12
3,Washington,37000,12
